!pip uninstall -y scikit-learn
!pip uninstall -y pandas

#!pip install scikit-learn==0.21.1
#!pip install pandas==0.24.2
#!pip3 install pandas_ml 
#!pip3 install pytorch_metric_learning 
!pip3 install faiss-cpu faiss-gpu

In [1]:
import os, sys
sys.path.append('pytorch_sol2/')
sys.path

['/floyd/home',
 '/usr/local/lib/python37.zip',
 '/usr/local/lib/python3.7',
 '/usr/local/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/xgboost-1.0.2-py3.7.egg',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython',
 'pytorch_sol2/']

import torch
torch.__version__

In [2]:
import pandas as pd
import numpy as np
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from skimage import io, transform
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch.nn.functional as F

from config import *
from GalaxiesDataset import *
from rsa_loader import *
from efigi_loader import *
from pytorchtools import EarlyStopping
from myEfigiLSCE import *

import pickle
from label_smoothing import LabelSmoothingCrossEntropy
import warnings
warnings.filterwarnings('ignore')

from pytorch_metric_learning import losses, miners, distances, reducers, testers, regularizers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

In [3]:
transf = transforms.Compose([transforms.Resize((105, 105)),
                             transforms.RandomHorizontalFlip(p=0.5),
                             transforms.RandomRotation(degrees=(0,360)),
                             transforms.RandomVerticalFlip(p=0.5),
                             transforms.ToTensor(),
                             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225])])

transformations = transforms.Compose([transforms.Resize((105, 105)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
#efigi_dir = base_path + '/datasets/efigi/pics/png/'
#
#efigi_csv = base_path + '/classes/efigi_model.csv
#'

In [5]:
train_ds = GalaxiesDataset_efigi(EFIGI_DIR, EFIGI_CSV, transform=transformations)

In [6]:
#def change(col):
cols = train_ds.classes_frame.columns.tolist()[3:]
K = 9
epsilon = 0.1
for col in cols:
    allz = np.where(train_ds.classes_frame.loc[:, col] == 0)[0].tolist()
    allo = np.where(train_ds.classes_frame.loc[:, col] == 1)[0].tolist()
    train_ds.classes_frame.loc[allz, col] = epsilon/(K-1)
    train_ds.classes_frame.loc[allo, col] = 1-epsilon
train_ds.classes_frame.head(5)

,pgc_name,full_pgc_name,hubb,E,Irr,S0,SBa,SBb,SBc,Sa,Sb,Sc
0,PGC 281,PGC0000281,Sa,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.9000,0.0125,0.0125
1,PGC 282,PGC0000282,SBc,0.0125,0.0125,0.0125,0.0125,0.0125,0.9000,0.0125,0.0125,0.0125
2,PGC 1841,PGC0001841,SBa,0.0125,0.0125,0.0125,0.9000,0.0125,0.0125,0.0125,0.0125,0.0125
3,PGC 3584,PGC0003584,Sb,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.9000,0.0125
4,PGC 635,PGC0000635,SBa,0.0125,0.0125,0.0125,0.9000,0.0125,0.0125,0.0125,0.0125,0.0125


In [7]:
cl = []
for row in range(train_ds.classes_frame.shape[0]):
    cl.append(np.argmax(train_ds.classes_frame.loc[:,train_ds.classes_frame.columns[3:]].loc[row]))

temp = pd.DataFrame(cl,columns=['cat'])
temp['cat'] = temp['cat'].astype('category')
temp['cat'] = temp['cat'].cat.codes


In [8]:
y = torch.from_numpy(temp.values)
train_idx, valid_idx= train_test_split(
np.arange(len(y)),
test_size=0.4,
shuffle=True,
    random_state=42,
stratify=y)

valid_idx, test_idx= train_test_split(
np.arange(len(y[valid_idx])),
test_size=0.5,
shuffle=True,
    random_state=42,
stratify=y[valid_idx])

In [9]:
size = len(train_ds)
indices = list(range(size))
split = int(np.floor(VALIDATION_SPLIT * size))
if SHUFFLE_DS:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

test_split = int(np.floor(VALIDATION_SPLIT * len(train_indices)))
if SHUFFLE_DS:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = train_indices[test_split:], train_indices[:test_split]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler   = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=8,
                                                 sampler=train_sampler)
val_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=8,
                                                 sampler=val_sampler)
unseen_dl = DataLoader(train_ds,batch_size=BATCH_SIZE, num_workers=8,
                                                 sampler=test_sampler)
print("Total: {} Train_dl: {} Validation_dl: {} Unseen_dl: {}".format(size, len(train_dl),                                                                    
                                                              len(val_dl), len(unseen_dl)))

Total: 3122 Train_dl: 6 Validation_dl: 2 Unseen_dl: 2


In [10]:
device = torch.device(DEVICE)

In [11]:
#Change to 'val' if only want to evaluate
train_or_val = 'val'

In [12]:
if train_or_val == 'train':
    model = torch.load('./models/gz2_resnet50')
    del model.fc[4]
    del model.fc[3]
    del model.fc[2]
    layers = []
    layers.append(model.fc[0])
    layers.append(model.fc[1])
    layers.append(nn.Linear(512, 128))
    del model.fc
    model.fc = nn.Sequential(*layers)
elif train_or_val == 'val':
    model = torch.load('./models/efigi_label_smoothin_128')    
    del model.fc[6]
    del model.fc[5]
    del model.fc[4]

In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.05, nesterov=True, weight_decay=0.0005)
criterion = LabelSmoothingCrossEntropy(reduction='mean')
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=3,threshold=0.0001,factor=0.1, verbose=True)

In [14]:
efigiLSCE = myEfigiLSCE(train_dl, val_dl, unseen_dl, model, optimizer, scheduler, criterion,device=device,BATCH_SIZE=BATCH_SIZE)
if train_or_val == 'train':
    efigiLSCE.train(n_epochs=300)

In [15]:
test_ds = GalaxiesDataset_rsa(TEST_DIR, TEST_CSV, transform=transformations)
#Label smoothing ...
cols = test_ds.classes_frame.columns.tolist()[2:]
K = 9
epsilon = 0.1
for col in cols:
    allz = np.where(test_ds.classes_frame.loc[:, col] == 0)[0].tolist()
    allo = np.where(test_ds.classes_frame.loc[:, col] == 1)[0].tolist()
    test_ds.classes_frame.loc[allz, col] = epsilon/(K-1)
    test_ds.classes_frame.loc[allo, col] = 1-epsilon
test_ds.classes_frame.head(5)
size = len(test_ds)
indices = list(range(size))
test_sampler = SubsetRandomSampler(indices)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=4,
                                                 sampler=test_sampler)
print("Total: {} Test_dl: {}".format(size, len(test_dl)))

Total: 1249 Test_dl: 4


In [16]:
res = efigiLSCE.RSAtest(test_dl)

Evaluation on RSA test set beginning:
Accuracy on RSA test set: 24.26
MAP@R on RSA test set: 0.2993
